# Bringing it all together

Let's do two exercises to incorporate the introductory and the regionalization aspects together.

## Exercise 1

Add a new ingredient to your meal; if that inventory isn't in a great state, you can use the default meal using the code below.

You will need to add sunflower seeds or oil, with a water input. Look at the extension table data to find a suitable (more or less reasonable) location.

Then, add the extension table raster `sunflowers.tiff` from the `data` folder. See notebook 1 on how to create a geocollection and notebook 6 on how to import extension tables (you only need the function `bwr.raster_as_extension_table()`, but it will be run multiple times).

Finally, copy and modify the configuration in notebook 7 to include your new extension table, and run your new LCA.

## Exercise 2

Make the following changes to your meal database.

In a **combinatorial** datapackage (**A**), add the following:

* Create two scenarios for the location of one ingredient, 
* Create two scenarios for that same ingredient, varying the amount of water consumed

In another datapackage (**B**):

Create new node (use the `id` 1000000); this will be "Eating the meal". We will create this node **only in the datapackage**. Create an input of 1 `meal`, but add significant uncertainty.

In the definition of your foreground inventory (i.e. in the database, not in a new datapackage), add uncertainty to at least two other ingredients which haven't been otherwise modified. This uncertainty can be on the edges where this ingredient is consumed, or on the edges where this ingredient consumes water or some other input.

Similarly, load the LCIA impact category `("water stress",)`, and change each characterization factor to a normal distribution with the mean having the same amount as before, but the $\sigma$ having a value of 0.1. Make sure you set both `amount` and `loc`. So this impact category is usable in both static and stochastic calculations.

Finally, pick a location being used by one of your water-consuming activities, and create a new datapackage (**C**) with 1000 possible values of the characterization factor in that region. This is a little tricky, there is an example of something similar below.

After you have made all these changes, do a LCA of this new system **using only the population density extension table** for all activities. This is because it isn't yet so easy to get the same values under Monte Carlo when creating multiple regionalized LCA instances.

If you are up for a real challenge, you can try to use multiple extensions tables for the respective ingredients as before. You will need to precompute (i.e. as before with a separate LCA class instance each time) and save copies of the following matrices **for each activity limitations possibility**, and then after each Monte Carlo iteration replace the values in the `LCA` class **for each activity limitations possibility**, summing the total.

* `LCA.inv_mapping_matrix`
* `LCA.distribution_normalization_matrix`
* `LCA.distribution_matrix`
* `LCA.xtable_matrix`

There are some hints below on how to create and use the datapackages in combination with your LCA class. Please make sure that the order of **A**, **B**, and **C** is correct, if they ave values that conflict with each other (otherwise it doesn't matter). The last one will take precendence, unless you chose to [override the default policies](https://github.com/brightway-lca/bw_processing#policies).

In [ ]:
import bw2io as bi

This project archive also available at [file.brightway.dev](https://files.brightway.dev/meal.tar.gz).

In [ ]:
bi.restore_project_directory("/srv/data/projects/meal.tar.gz", overwrite_existing=True)

Here is an example of using a datapackage in regionalized LCA:

In [ ]:
import bw2data as bd
import bw2regional as bwr
import bw_processing as bwp
import numpy as np

In [ ]:
PROJECT_NAME = "Spain case study"

In [ ]:
bd.projects.set_current(PROJECT_NAME)

In [ ]:
meal = bd.get_node(name="meal")

In [ ]:
h2o = bd.get_node(name="water")

In [ ]:
reg_cfs = bd.Method(("water stress",)).load()

In [ ]:
reg_cfs[:10]

In [ ]:
big_numbers = bwp.create_datapackage()

We will write big numbers for our characterization factors. This is mostly just to test to make sure that we can do what we are trying to do.

We need to change the `characterization_matrix`. This matrix is a bit tricky, see the [documentation](https://github.com/brightway-lca/brightway2-regional/blob/70a37c2804243d0d0d50ce208d892a5af281b122/bw2regional/lca/base_class.py#L120). Our first index will be the elementary flow, our second index will be the location. We need to look up the *integer id* of the locations; we do this using `bw2data.geomapping`.

In [ ]:
indices = np.array(
    [(h2o.id, bd.geomapping[z]) for _, _, z in reg_cfs],
    dtype=bwp.INDICES_DTYPE
)

In [ ]:
big_numbers.add_persistent_vector(
    matrix='characterization_matrix',
    indices_array=indices,
    data_array=np.random.normal(loc=1000, scale=100, size=len(indices)),
)

In [ ]:
lca = bwr.TwoSpatialScalesLCA({meal: 1}, ("water stress",), extra_data_objs=[big_numbers])
lca.lci()
lca.lcia()
lca.score